In [2]:
import pandas as pd
from itertools import product
import yfinance as yf
import plotly.graph_objects as go

from sklearn.linear_model import LinearRegression

from marketprediction import config

In [4]:
ticker = 'AAPL'
df = pd.read_excel("../data/data.xlsx", ticker).set_index('Date')
df

,Close,High,Low,Open,Volume
Date,,,,,
2010-01-04,6.424605,6.439315,6.375673,6.407194,493729600
2010-01-05,6.435714,6.472038,6.401791,6.442318,601904800
2010-01-06,6.333344,6.461230,6.326740,6.435713,552160000
2010-01-07,6.321636,6.364265,6.275706,6.356760,477131200
2010-01-08,6.363664,6.364264,6.276006,6.313230,447610800
...,...,...,...,...,...
2024-12-24,257.286682,257.296626,254.386957,254.586262,23234700
2024-12-26,258.103729,259.179926,256.718662,257.276679,27237100
2024-12-27,254.685867,257.784882,252.164818,256.917934,42355300


In [4]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Date'], y=df['Close']))
fig.update_layout(title=ticker)
fig.show()

In [ ]:
class Strategy:
    def generate_signals(self, data):
        """Return a column 'signal' : 1 -> buy ; 0 -> hold ; -1 -> sell"""
        raise NotImplementedError("Each Strategy must implement generate_signals")
    

class MovingAverageStrategy(Strategy):
    def __init__(self, short=5, long=30, thresh=0.9):
        self.short = short
        self.long = long
        self.thresh = thresh

    def generate_signals(self, data):
        data = data.copy()
        data['MA_short'] = data['Close'].shift(1).rolling(window=self.short).mean()
        data['MA_long'] = data['Close'].shift(1).rolling(window=self.long).mean()
        data['signal'] = 0
        data.loc[(data['MA_short'] < self.thresh * data['MA_long']) & (data['MA_short'].shift(1) >= self.thresh * data['MA_long'].shift(1)), 'signal'] = 1
        
        return data
    

class Backtester:
    def __init__(self, data, strategy, initial_capital=1000):
        self.data = data
        self.strategy = strategy
        self.initial_capital = initial_capital
        self.trades = []

    def run(self):
        df = self.strategy.generate_signals(self.data)
        position = None
        entry_price = None
        for idx, row in df.iterrows():
            if position is None & row['signal'] == 1:
                position = "long"
                entry_price = row['Open']
                entry_idx = idx
            elif position is not None:
                if (idx - entry_idx).days >= 5:
                    exit_price = row['Close']
                    self.trades.append({"entry_idx": entry_idx, "exit_idx": idx, "entry_price": entry_price, "exit_price": exit_price, "return": (exit_price - entry_price) / entry_price})
                    position = None
        return self.trades

    